# This demonstrates the following on imdb data
Use GloVe model for embedding (without learning) instead of keras embedding (that leanrs)
1) Create a LSTM model with return_sequence = False for the last layer
2) Learn a WordEmbedding using word2vec (download weights, and train using additional sample sentences) using both skipgram and CBoW
3) Predict sample sentences using both approach 1 and 2
4) Do some analogies calculus using glove and word2vec

In [ ]:
import keras

from keras.datasets import imdb
from keras.layers import Dense,  Flatten
from keras.layers import Conv1D, MaxPooling1D

In [ ]:
vocabulary_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('IMDB dataset  {} training samples, {} test samples'.format(len(X_train), len(X_test)))

In [ ]:
print('---Features: review in number sequence After padding---')
print(X_train[6])
print('Sentiment 1 = positve, 0 = negative')
print(y_train[6])

In [ ]:
w2id = imdb.get_word_index() 
id2word = {i: word  for word, i in w2id.items()}
print (len(w2id), id2word[1], w2id["the"])

In [ ]:
#Jibberish data?
actual_data = []
for i in range (len(X_train[6])):
    ind = X_train[6][i]
    if(ind != 0):
        actual_data.append(id2word[ind])
actual_data = " ".join (actual_data )
print (actual_data, "\nAnd sentiment is ", y_train[6])

In [ ]:
word2id ={w: i+3 for w, i in w2id.items()}
word2id["__PADDING__"] = 0
word2id["__START__"] = 1
word2id["__UNK__"] = 2

#This returns the index of the words from 1 to n with 1 being the most frequently occuring word, 

# and n the least frequently occuring word

print (type(word2id), len (word2id))

id2word = {i: word  for word, i in word2id.items()}
print(id2word[1], id2word[2], #id2word[3],#
      id2word[4], id2word[5], id2word[6], id2word[88586])
actual_data = []
for i in range (len(X_train[6])):
    ind = X_train[6][i]
    if(ind != 0):
        actual_data.append(id2word[ind])
actual_data = " ".join (actual_data )
print (actual_data, "\nAnd sentiment is ", y_train[6])
print (word2id["the"])

In [16]:
print('Maximum review length: {}'.format(
len((max((X_train + X_test), key=len)))))
import numpy as np

Maximum review length: 2697


In [19]:
def read_glove_data(glove_file):
    with open(glove_file, 'r', encoding="utf-8") as f:

        word_to_vec_map = {}
        curr_word=None
        WordToInd = {}
        IndToWord = {}
        i = -1
        try:
            for line in f:
                i+=1
                try:
                    line = line.strip().split()
                    curr_word = line[0]                   
                   
                    word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
                    WordToInd[curr_word] = i
                    IndToWord[i] = curr_word
                except Exception as E:
                    print ("got An exception, word=", curr_word, i)
                    pass         
        except Exception as E:
            print ("got An exception before for, word=", curr_word, i)
            pass         
            
    return word_to_vec_map, WordToInd, IndToWord

In [20]:
word_to_vec_map, WordToInd, IndToWord = read_glove_data('glove.6B.50d.txt')

In [43]:
#Now create an embedding vector
gl_embed = np.zeros((vocabulary_size, 50))
for i, word in id2word.items():
    if i >= vocabulary_size:
        continue
    embed = word_to_vec_map.get(word)
    if embed is not None:
        gl_embed[i] = embed

In [45]:
print (gl_embed.shape)
print (gl_embed[100], gl_embed[10], gl_embed[0] )

(10000, 50)
[ 0.90754   -0.38322    0.67648   -0.20222    0.15156    0.13627
 -0.48813    0.48223   -0.095715   0.18306    0.27007    0.41415
 -0.48933   -0.0076005  0.79662    1.0989     0.53802   -0.54468
 -0.16063   -0.98348   -0.19188   -0.2144     0.19959   -0.31341
  0.24101   -2.2662    -0.25926   -0.10898    0.66177   -0.48104
  3.6298     0.45397   -0.64484   -0.52244    0.042922  -0.16605
  0.097102   0.044836   0.20389   -0.46322   -0.46434    0.32394
  0.25984    0.40849    0.20351    0.058722  -0.16408    0.20672
 -0.1844     0.071147 ] [-0.17168   0.3855    0.50195  -0.58935  -1.0504    0.31327   0.12463
 -1.36     -1.2506   -0.94656   0.28644   1.2532   -0.54125  -0.25494
 -0.69635   0.15366  -0.9228    0.70842  -0.38708   0.45516  -0.26158
 -0.82918   0.45284   0.46074  -0.60629   0.31825  -0.56607   0.19254
 -0.22289   0.15175   0.37615   0.15187   0.78425   1.1723    0.44189
 -0.17558   0.83528  -0.76923  -0.19997   0.50355   0.70561  -0.31922
  0.20774   0.030179 -0.

In [29]:
from keras.preprocessing import sequence
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
modelLSTM=None

In [30]:
batch_size = 256
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

In [46]:
#Lets now create an LSTM model with embedding layer.
#Set the embedding layer as not trainable, and sets its weights
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras import Sequential
embedding_size=50
modelLSTM=Sequential()
modelLSTM.add(Embedding(vocabulary_size, embedding_size, input_length=max_words,
                       weights=[gl_embed], trainable=False))
modelLSTM.add(LSTM(128, dropout=0.2, return_sequences=False)) 
modelLSTM.add(Dense(1, activation='sigmoid'))
print(modelLSTM.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 50)           500000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 591,777
Trainable params: 91,777
Non-trainable params: 500,000
_________________________________________________________________
None


In [47]:
modelLSTM.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [48]:
modelLSTM.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Train on 24744 samples, validate on 256 samples
Epoch 1/3
24744/24744 [==============================] - 602s 24ms/step - loss: 0.6393 - acc: 0.6303 - val_loss: 0.5644 - val_acc: 0.7148
Epoch 2/3
24744/24744 [==============================] - 592s 24ms/step - loss: 0.5972 - acc: 0.6864 - val_loss: 0.4757 - val_acc: 0.8047
Epoch 3/3
24744/24744 [==============================] - 585s 24ms/step - loss: 0.5668 - acc: 0.7096 - val_loss: 0.4632 - val_acc: 0.8047


In [49]:
scores = modelLSTM.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.75972


In [ ]:
#Lets now create an LSTM model with embedding layer.
#Set the embedding layer as trainable, and sets its weights
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras import Sequential
embedding_size=50
modelLSTM1=Sequential()
modelLSTM1.add(Embedding(vocabulary_size, embedding_size, input_length=max_words,
                       weights=[gl_embed]))
modelLSTM1.add(LSTM(128, dropout=0.2, return_sequences=False)) 
modelLSTM1.add(Dense(1, activation='sigmoid'))
print(modelLSTM1.summary())
modelLSTM1.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
modelLSTM1.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

In [ ]:
#Lets now create an LSTM model with keras embedding layer and let it learn.
#Set the embedding layer as not trainable, and sets its weights
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras import Sequential
embedding_size=50
modelLSTM2=Sequential()
modelLSTM2.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
modelLSTM2.add(LSTM(128, dropout=0.2, return_sequences=False)) 
modelLSTM2.add(Dense(1, activation='sigmoid'))
print(modelLSTM2.summary())
modelLSTM2.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
modelLSTM2.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

In [50]:
reviewList = ["the movie was not at all bad", 
              "the movie was a total waste of my time",
              "the food was so deliciously delicious that i felt sinfully wicked" ,   
              "the food was so wicked that after eating i felt sinful"
             ]
print ("Now going to predict these reviews")

Now going to predict these reviews


In [ ]:
print (word2id["the"])

In [51]:
def PredictSentiment(reviewList, ModelListTuple):
    sentiment= {True: "Positive",
               False: "Negative"}
    Threshold = 0.5
    for r in reviewList:
        words = r.split()
        review = []
        print ("review=", r)
        for word in words:
          if word not in word2id: 
            review.append(2)
            print (word, "Appended 2")
          else:
            if (word2id[word]) >= vocabulary_size:
                print("got a word outside teh vocab_index", word, word2id[word], "breaking")
                break
            print ("word=", word, "id=", word2id[word])
            review.append(word2id[word]) 
        review = keras.preprocessing.sequence.pad_sequences([review],
          truncating='pre', padding='pre', maxlen=max_words)
        for i,m in enumerate(ModelListTuple):
            if m[0] is not None:
                prediction = m[0].predict(review)
                print("Prediction Probability for ", "\"",r, "\"",ModelListTuple[i][1]," = ", prediction[0][0], "Sentiment=", 
                      sentiment[prediction[0][0]>Threshold], "\n")
                
PredictSentiment(reviewList,[ (modelLSTM, "LSTM")])

review= the movie was not at all bad
word= the id= 4
word= movie id= 20
word= was id= 16
word= not id= 24
word= at id= 33
word= all id= 32
word= bad id= 78
Prediction Probability for  " the movie was not at all bad " LSTM  =  0.3426385 Sentiment= Negative 

review= the movie was a total waste of my time
word= the id= 4
word= movie id= 20
word= was id= 16
word= a id= 6
word= total id= 964
word= waste id= 437
word= of id= 7
word= my id= 61
word= time id= 58
Prediction Probability for  " the movie was a total waste of my time " LSTM  =  0.3150782 Sentiment= Negative 

review= the food was so deliciously delicious that i felt sinfully wicked
word= the id= 4
word= food id= 1644
word= was id= 16
word= so id= 38
word= deliciously id= 6922
word= delicious id= 6335
word= that id= 15
word= i id= 13
word= felt id= 421
sinfully Appended 2
word= wicked id= 3799
Prediction Probability for  " the food was so deliciously delicious that i felt sinfully wicked " LSTM  =  0.46510503 Sentiment= Negative 


In [ ]:
from keras.models import Model
model = modelCNN
layer_outputs = [layer.output for layer in model.layers]
i = 0
print ("#Layers = ", len(layer_outputs))
print (type(model.layers[0]))
l = model.layers[0]
print (len(l.get_weights()))
print (l.get_weights()[0].shape)
print (word2id["apple"], word2id["orange"], id2word[6335], id2word[6922])
print (l.get_weights()[0][word2id["apple"]])
print (l.get_weights()[0][word2id["orange"]])

In [ ]:
#Now use patience, and early stopping
#early1 = keras.callbacks.EarlyStopping(monitor ="val_acc", patience=1, restore_best_weights=True)
#early1 = keras.callbacks.EarlyStopping(monitor ="acc", patience=2, baseline = 0.95)
#early1 = keras.callbacks.EarlyStopping(monitor ="val_acc", patience=1, restore_best_weights=True)

callback_list = [#early1
                keras.callbacks.ModelCheckpoint(filepath="my_mod2.h5", monitor="val_acc",
                                               save_best_only=True),
                #keras.callbacks.TerminateOnNaN()
                ]

In [ ]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
X_train3 = X_train2[:1000]
Y_train3 = y_train2[:1000]
print (X_train3.shape, Y_train3.shape)

In [ ]:
model.fit(X_train3, Y_train3, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=15, #num_epochs,
             callbacks=callback_list)